In [16]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass
import pdvega
import time
import random
from configobj import ConfigObj
from glob import glob

In [17]:
# Create a posgresql database connection using settings from config file
# Create a database connection using settings from config file
config='../db/config.ini'
# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = '243'
    conn_info["sqlhost"] = 'localhost'
    conn_info["sqlport"] = 5432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '243':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == 'localhost') & (conn_info["sqlport"]=='5432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres
Password: ········


In [72]:
df=pd.read_csv(os.getcwd()+'\\allpatientdata.csv')

In [73]:
def main():
    start = time.clock()
    ftName_0 = ['uniquepid','patienthealthsystemstayid','patientunitstayid','unitvisitnumber','gender',
                'age','height','weight','LoS','Firstday']
 
    df_P = pd.DataFrame(columns= ftName_0)
    th = 0
    for i in range(0,len(df)):
        print('%d/%d:%s'%(i,len(df),df['patientunitstayid'][i]),end='\r')
        df_P = patient(df['patientunitstayid'][i], th, df_P, ftName_0)
        th=th+1
    df_P.to_csv(os.getcwd()+'\\allpatientfile.csv',index=False)
    end = time.clock() - start
    print(end)

In [74]:
#Create Patient file
def patient(patientunitstayid,th,df_P,ftName_0):
    query = query_schema + """
    select *
    from patient
    where patientunitstayid='{}'
    order by uniquepid,patient.patientunitstayid
    """.format(patientunitstayid)
    df_patient = pd.read_sql_query(query, con)
    df_patient = df_patient[['uniquepid','patienthealthsystemstayid','patientunitstayid','unitvisitnumber','gender','age','admissionheight','admissionweight', 'dischargeweight',
                             'hospitaladmitoffset', 'hospitaldischargeoffset', 'hospitaladmittime24', 'hospitaldischargetime24']]

    #print(df_patient)
    for i in range (0,len(df_patient.columns)):
        if df_patient.columns[i] == 'admissionheight':
            tempC = list(df_patient.columns)
            tempC[i] = 'height'
            df_patient.columns = tempC
        elif df_patient.columns[i] == 'admissionweight':
            tempC = list(df_patient.columns)
            tempC[i] = 'weight'
            df_patient.columns = tempC
    for i in range(0,len(df_patient.index)):  
        if pd.isnull(df_patient['weight'][i]):
            df_patient['weight'][i] = df_patient['dischargeweight'][i]
    for i in range(0,len(df_patient.columns)):
        if df_patient.columns[i] == 'dischargeweight':
            break
    
    space_row = []
    for i in range(len(ftName_0)):
        space_row.append(-0.001)
    #計算總住院天數及第一天換日分鐘數
    #calculate the loS and turn the first "day" into "minute" 
    #space_columns = np.zeros((len(df.index),), dtype=int)
    a=[]
    b=[]
    for i in range(0,len(df_patient.index)):
        struct_time = time.strptime(df_patient['hospitaladmittime24'][i],'%H:%M:%S')
        hospitaladmittime = struct_time.tm_hour*60 + struct_time.tm_min + struct_time.tm_sec/60
        Firstday = df_patient['hospitaladmitoffset'][i] + (24*60 - hospitaladmittime)
        lenofstay = (df_patient['hospitaldischargeoffset'][i] - Firstday)/ (24*60) + 1
        if (lenofstay - int(lenofstay)) > 0:
            lenofstay = lenofstay + 1
        #print(int(lenofstay), Firstday)
        a.append(int(lenofstay))
        b.append(Firstday)
        #print("Done")
        df_patient.insert(len(df_patient.columns),"LoS",a)
        df_patient.insert(len(df_patient.columns),"Firstday",b)
    df_P = df_P.append(pd.Series(space_row, index=df_P.columns), ignore_index=True)
    for i in (ftName_0):
        df_P[i][th] = df_patient[i][0]  
    return df_P

In [75]:
if __name__ is '__main__':
    main()

c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\userpc\miniconda3\envs\243\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


111.51550400000997


c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]


In [78]:
#extract ESRD
df=pd.read_csv(os.getcwd()+'\\allpatientfile.csv')
d=[]
for i in range(0,len(df)):
    print("%d/%d:%s"%(i,len(df),df['patientunitstayid'][i]),end='\r')
    query = query_schema + """
        select * from diagnosis where patientunitstayid='{}' and diagnosisstring like '%ESRD%'  
        """.format(int(df['patientunitstayid'][i]))
    df_esrd = pd.read_sql_query(query, con)
    #print(df_dnr)
    if(len(df_esrd)==0):
        d.append(0)
    else:
        d.append(1)
df.insert(len(df.columns),'ESRD',d)
df.to_csv(os.getcwd()+'\\allpatientfile.csv',index=False)   

In [79]:
#extract Apache IV score
df=pd.read_csv(os.getcwd()+'\\allpatientfile.csv')
d=[]
for i in range(0,len(df)):
    print("%d/%d:%s"%(i,len(df),df['patientunitstayid'][i]),end='\r')
    query = query_schema + """
        select apachescore
        from apachePatientResult
        where patientunitstayid = '{}'
        """.format(int(df['patientunitstayid'][i]))
    df_apc = pd.read_sql_query(query, con)
    df_apc=df_apc[['apachescore']]
    #print(df_apc)
    if(len(df_apc)==0 or df_apc['apachescore'][0]==-1):
        d.append(random.randint(20,30))
        continue
    else:
        d.append(df_apc['apachescore'][0])
df.insert(len(df.columns),'Apachescore',d)
df.to_csv(os.getcwd()+'\\allpatientfile.csv',index=False)   

In [80]:
df=pd.read_csv(os.getcwd()+'\\allpatientfile.csv')
c=[]
d=[]
for i in range(0,len(df)):
    print('%d/%d:%s'%(i,len(df),df['patientunitstayid'][i]),end='\r')
    query = query_schema + """
    select * from patient where patientunitstayid='{}'
    """.format(int(df['patientunitstayid'][i]))
    df_patient = pd.read_sql_query(query, con)
    df_patient=df_patient[['hospitaldischargeoffset','hospitaldischargestatus']]
    c.append(df_patient['hospitaldischargeoffset'][0])
    d.append(df_patient['hospitaldischargestatus'][0])
df.insert(len(df.columns),'hospitaldischargeoffset',c)
df.insert(len(df.columns),'hospitaldischargestatus',d)
df.to_csv(os.getcwd()+'\\allpatientfile.csv',index=False)

In [83]:
df=pd.read_csv(os.getcwd()+'\\allpatientfile.csv')
unabridgedactualventdays=[]
ethnicity=[]
for i in range(0,len(df)):
    print('%d/%d:%s'%(i,len(df),df['patientunitstayid'][i]),end='\r')
    ventday=[]
    query = query_schema + """
    select * from patient,apachepatientresult where 
    patient.patientunitstayid=apachepatientresult.patientunitstayid and
    patient.patientunitstayid = '{}'
    """.format(int(df['patientunitstayid'][i]))
    df_patient = pd.read_sql_query(query, con)
    df_patient = df_patient[['unabridgedactualventdays','ethnicity']]
    if(len(df_patient)!=0):
        if(not pd.isna(df_patient['unabridgedactualventdays'][0])):
            unabridgedactualventdays.append(df_patient['unabridgedactualventdays'][0])
        if(not pd.isna(df_patient['ethnicity'][0])):
            ethnicity.append(df_patient['ethnicity'][0])
        if(pd.isna(df_patient['unabridgedactualventdays'][0])):
            unabridgedactualventdays.append('NULL')
        if(pd.isna(df_patient['ethnicity'][0])):
            ethnicity.append('Other/Unknown')
    else:
        unabridgedactualventdays.append('NULL')
        ethnicity.append('Other/Unknown')
df.insert(len(df.columns),'unabridgedactualventdays',unabridgedactualventdays)
df.insert(len(df.columns),'ethnicity',ethnicity)
df.to_csv(os.getcwd()+'\\allpatientfile.csv',index=False)

In [84]:

df=pd.read_csv(os.getcwd()+'\\allpatientfile.csv')
#unabridgedactualventdays=[]
actualICULOS=[]
for i in range(0,len(df)):
    print('%d/%d:%s'%(i,len(df),df['patientunitstayid'][i]),end='\r')
    ventday=[]
    query = query_schema + """
    select * from patient,apachepatientresult where 
    patient.patientunitstayid=apachepatientresult.patientunitstayid and
    patient.patientunitstayid = '{}'
    """.format(int(df['patientunitstayid'][i]))
    df_patient = pd.read_sql_query(query, con)
    df_patient = df_patient[['actualiculos']]
    if(len(df_patient)!=0):
        if(not pd.isna(df_patient['actualiculos'][0])):
            actualICULOS.append(df_patient['actualiculos'][0])
        if(pd.isna(df_patient['actualiculos'][0])):
            actualICULOS.append('NULL')
    else:
        actualICULOS.append('NULL')
df.insert(len(df.columns),'actualICULOS',actualICULOS)
df.to_csv(os.getcwd()+'\\allpatientfile.csv',index=False)